## **PREPROCESSING DATA FILM**

**MEMUAT DATA**

In [10]:
import pandas as pd
import numpy as np

# 1. Memuat Data (Load)
# ------------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/data_mining/movie_sample_dataset.csv'
df = pd.read_csv(file_path)
print(">>> Dataset berhasil dimuat dari Google Drive.\n")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
>>> Dataset berhasil dimuat dari Google Drive.



Proses ini diawali dengan mengimpor library yang esensial, yaitu Pandas untuk manipulasi data dan NumPy untuk komputasi numerik. Selanjutnya, skrip ini menghubungkan Google Colab ke Google Drive menggunakan drive.mount agar file di dalamnya dapat diakses. Setelah koneksi berhasil, lokasi spesifik dari file dataset disimpan dalam variabel file_path, yang kemudian digunakan oleh fungsi pd.read_csv dari Pandas untuk membaca data dari file CSV dan memuatnya ke dalam sebuah tabel data yang disebut DataFrame dengan nama df. Terakhir, sebuah pesan konfirmasi dicetak untuk menandakan bahwa seluruh proses pemuatan data telah berhasil diselesaikan.


**MEMERIKSA DATA**

In [11]:
# 2. Memeriksa Data (Inspect)
# ------------------------------------------------------------------------------
print("[Info Awal] 50 Baris Pertama:")
print(df.head(50))
print("\n[Info Awal] Informasi Umum Dataset:")
# replace "" menjadi NaN
df.replace("N/A", np.nan, inplace = True)
df.replace("Null", np.nan, inplace = True)
df.replace("Nan", np.nan, inplace = True)
df.info()
# cek banyaknya missing value pada masing-masing kolom
print(df.isnull().sum())

[Info Awal] 50 Baris Pertama:
               color         director_name  duration        gross  \
0              Color       Martin Scorsese       240  116866727.0   
1              Color           Shane Black       195  408992272.0   
2             color      Quentin Tarantino       187   54116191.0   
3              Color      Kenneth Lonergan       186      46495.0   
4              Color         Peter Jackson       186  258355354.0   
5                NaN                   NaN       183  330249062.0   
6              Color         Peter Jackson       -50  303001229.0   
7              Color           Edward Hall       180          NaN   
8              Color           Joss Whedon       173  623279547.0   
9              Color           Joss Whedon       173  623279547.0   
10               NaN            Tom Tykwer       172   27098580.0   
11             Color                  Null       158  102515793.0   
12             Color   Christopher Spencer       170   59696176.0   
13  

Pada tahap ini, proses dimulai dengan inspeksi data secara visual dengan menampilkan 50 baris pertama menggunakan df.head(50) untuk mendapatkan gambaran umum. Sebelum melakukan pengecekan data kosong, dilakukan langkah pembersihan awal di mana berbagai representasi teks dari nilai kosong seperti "N/A", "Null", dan "Nan" diseragamkan menjadi format standar np.nan agar dapat dideteksi oleh Pandas. Setelah itu, df.info() digunakan untuk melihat ringkasan teknis dari setiap kolom, termasuk tipe data dan jumlah nilai non-null. Terakhir, df.isnull().sum() dijalankan untuk menghitung dan menampilkan jumlah total nilai kosong (missing value) yang sebenarnya ada di setiap kolom setelah proses standarisasi.

Berikut info yang diperoleh dari info awal.
Dalam dataset ini terdapat 13 kolom, yaitu color, director_name, duration, gross, genres, movie_title, title_year, language, country, budget, imdb_score, actors, dan movie_facebook_likes. Setiap kolom dapat diketahui tipe datanya, seperti kolom gross menggunakan tipe data float64. Selain itu, dapat diperoleh juga banyaknya baris dalam dataset tersebut adalah 100. Info lainnya yang diperoleh adalah banyaknya missing value yang ada di setiap kolomnya.


**PEMBERSIHAN DATA**

In [12]:
# 3. Membersihkan Data (Prinsip: Handle Missing Values & Inconsistencies)
# -----------------------------------------------------------------------------

print("\n\n--- Langkah 3: Membersihkan Data ---\n")

# Menghapus baris yang memiliki nilai NaN di kolom 'gross' atau 'budget'
df.dropna(subset=['gross', 'budget'], inplace=True)
print("[Pembersihan] Baris dengan nilai 'gross' atau 'budget' yang kosong telah dihapus.")

# Mengisi nilai kosong di kolom 'color' dengan nilai yang paling sering muncul (modus)
color_mode = df['color'].mode()[0]
df['color'].fillna(color_mode, inplace=True)
print(f"[Pembersihan] Nilai kosong di kolom 'color' telah diisi dengan modus ('{color_mode}').")

# Mengisi nilai kosong di 'genres' dengan 'unknown'
df['genres'].fillna('unknown', inplace=True)
print("[Pembersihan] Nilai 'genres' yang kosong telah diisi dengan 'unknown'.")

# Mengisi nilai kosong di 'director_name' dengan 'nameless'
df['director_name'].fillna('nameless', inplace=True)
print("[Pembersihan] Nilai 'director_name' yang kosong telah diisi dengan 'nameless'.")

# Memperbaiki kesalahan input
corrections = {205: 2015, 202: 2012} # Dictionary untuk koreksi
df['title_year'] = df['title_year'].replace(corrections)
print(f"[Pembersihan] Kesalahan input tahun {list(corrections.keys())} telah diperbaiki.")
df['country'] = df['country'].replace('United States', 'USA')
print("[Pembersihan] Telah mengubah 'United States' menjadi 'USA'.")

# Standarisasi penulisan di kolom 'country' dan 'color' menjadi huruf kecil (prinsip: atasi inkonsistensi)
df['country'] = df['country'].str.strip().str.lower()
df['color'] = df['color'].str.strip().str.lower()
print("[Pembersihan] Kolom 'country' telah distandarisasi.")
print("[Pembersihan] Kolom 'color' telah distandarisasi (menjadi huruf kecil dan tanpa spasi).")

# Penanganan nilai tidak valid/tidak logis (termasuk nilai negatif)
initial_rows = len(df)
df = df[df['gross'] >= 0]
df = df[df['budget'] >= 0]
df = df[df['duration'] >= 60] # Durasi harus lebih dari 0
df = df[(df['imdb_score'] >= 1) & (df['imdb_score'] <= 10)]# Memastikan imdb_score berada di rentang valid (1-10)
print(f"[Pembersihan] {initial_rows - len(df)} baris dengan nilai tidak valid (negatif atau di luar rentang) telah dihapus.")

# Mengecek dan menampilkan baris duplikat sebelum dihapus
print("\n[Pengecekan Duplikat] Mengecek baris duplikat...")
duplicate_rows = df[df.duplicated(keep=False)]

if not duplicate_rows.empty:
    print(f"Ditemukan {len(duplicate_rows)} baris yang terlibat dalam duplikasi:")
    print(duplicate_rows)
else:
    print("Tidak ditemukan baris duplikat.")

# Menghapus baris duplikat
initial_rows = len(df)
df.drop_duplicates(inplace=True)
print(f"[Pembersihan] {initial_rows - len(df)} baris duplikat telah dihapus.")



--- Langkah 3: Membersihkan Data ---

[Pembersihan] Baris dengan nilai 'gross' atau 'budget' yang kosong telah dihapus.
[Pembersihan] Nilai kosong di kolom 'color' telah diisi dengan modus ('Color').
[Pembersihan] Nilai 'genres' yang kosong telah diisi dengan 'unknown'.
[Pembersihan] Nilai 'director_name' yang kosong telah diisi dengan 'nameless'.
[Pembersihan] Kesalahan input tahun [205, 202] telah diperbaiki.
[Pembersihan] Telah mengubah 'United States' menjadi 'USA'.
[Pembersihan] Kolom 'country' telah distandarisasi.
[Pembersihan] Kolom 'color' telah distandarisasi (menjadi huruf kecil dan tanpa spasi).
[Pembersihan] 4 baris dengan nilai tidak valid (negatif atau di luar rentang) telah dihapus.

[Pengecekan Duplikat] Mengecek baris duplikat...
Ditemukan 10 baris yang terlibat dalam duplikasi:
    color director_name  duration        gross                     genres  \
8   color   Joss Whedon       173  623279547.0    Action|Adventure|Sci-Fi   
9   color   Joss Whedon       173  6

/tmp/ipython-input-3961180642.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['color'].fillna(color_mode, inplace=True)
/tmp/ipython-input-3961180642.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

Tahap pembersihan data ini dimulai dengan menangani nilai yang hilang (missing values) melalui dua strategi: menghapus baris jika kolom vital seperti gross atau budget kosong, dan mengisi (imputation) nilai kosong pada kolom kategorikal seperti color, genres, dan director_name dengan nilai pengganti yang logis (modus atau unknown). Selanjutnya, dilakukan koreksi data untuk memperbaiki kesalahan input spesifik seperti tahun rilis dan menyeragamkan format penulisan pada kolom country dan color menjadi huruf kecil untuk konsistensi. Proses dilanjutkan dengan validasi data, di mana baris dengan nilai yang tidak masuk akal (seperti gross negatif atau imdb_score di luar skala 1-10) akan difilter dan dihapus. Terakhir, skrip melakukan pengecekan duplikasi dengan menampilkan baris-baris yang identik sebelum akhirnya menghapus duplikat tersebut untuk memastikan setiap entri dalam dataset bersifat unik.

Dari pengecekan data duplikat, terdapat 10 baris yang terlibat dalam duplikasi.

**TRANSFORMASI DATA**

In [13]:
# 4. Transformasi Data (Prinsip: Correct Data Types & Dummy Variables)
# -----------------------------------------------------------------------------

print("\n\n--- Langkah 4: Transformasi Data ---\n")

# Ubah tipe data kolom (Prinsip: Correct Data Types)
df['gross'] = df['gross'].astype(int)
df['budget'] = df['budget'].astype(int)
print("[Transformasi] Tipe data 'gross' dan 'budget' berhasil diubah menjadi integer.")

# Normalisasi umum untuk semua kolom teks lainnya
text_columns = ['director_name', 'movie_title', 'language', 'actors', 'genres']
for col in text_columns:
    df[col] = df[col].str.strip().str.lower()

print("[Normalisasi] Kolom teks lainnya ('director_name', 'movie_title', 'language', 'actors', 'genres') telah dinormalisasi (huruf kecil).")

# Membuat variabel dummy untuk 'genres' (Prinsip: Dummy Variables)
genre_dummies = df['genres'].str.get_dummies(sep='|')
df = pd.concat([df, genre_dummies], axis=1)

# Menghapus kolom asli setelah diubah menjadi dummy
df.drop('genres', axis=1, inplace=True)
print(f"[Transformasi] Kolom 'genres' dipecah menjadi {len(genre_dummies.columns)} kolom dummy.")




--- Langkah 4: Transformasi Data ---

[Transformasi] Tipe data 'gross' dan 'budget' berhasil diubah menjadi integer.
[Normalisasi] Kolom teks lainnya ('director_name', 'movie_title', 'language', 'actors', 'genres') telah dinormalisasi (huruf kecil).
[Transformasi] Kolom 'genres' dipecah menjadi 18 kolom dummy.


Pada tahap transformasi data ini, proses diawali dengan memperbaiki tipe data dengan mengubah kolom gross dan budget menjadi format integer agar sesuai dengan sifatnya sebagai bilangan bulat. Setelah itu, dilakukan normalisasi teks di mana beberapa kolom tekstual seperti director_name dan movie_title diseragamkan dengan diubah menjadi huruf kecil dan dibersihkan dari spasi berlebih untuk memastikan konsistensi. Puncak dari langkah ini adalah transformasi kolom genres menjadi variabel dummy; setiap genre unik yang dipisahkan oleh tanda | dipecah menjadi kolomnya sendiri yang berisi nilai biner (0 atau 1), dan setelah itu, kolom genres yang asli dihapus karena informasinya telah berhasil direpresentasikan dalam format baru yang siap untuk analisis.

**VERIFIKASI DATA DAN MENYIMPAN DATA**

In [14]:
# 5. Verifikasi dan Penyimpanan (Prinsip: Final Check & Save)
# -----------------------------------------------------------------------------

print("\n\n--- Verifikasi Akhir dan Penyimpanan ---\n")

print("[Hasil Akhir] 5 Baris Pertama Data yang Sudah Diproses:")
# Menampilkan beberapa kolom untuk menunjukkan perubahan, termasuk kolom genre baru
display_columns = ['movie_title', 'gross', 'budget', 'color'] + list(genre_dummies.columns[:3])
print(df[display_columns].head())

print("\n[Hasil Akhir] Informasi Dataset Final:")
df.info()

#Memeriksa keberadaan cell yang kosong
# cek banyaknya missing value pada masing-masing kolom
print(df.isnull().sum())

# Menyimpan file hasil ke Google Drive
output_path = '/content/drive/MyDrive/data_mining/movie_dataset_cleaned.csv'
df.to_csv(output_path, index=False)

print(f"\n>>> Proses Selesai! Data bersih telah disimpan di Google Drive sebagai '{output_path}' <<<")



--- Verifikasi Akhir dan Penyimpanan ---

[Hasil Akhir] 5 Baris Pertama Data yang Sudah Diproses:
                           movie_title      gross     budget  color  action  \
0              the wolf of wall street  116866727  100000000  color       0   
1                           iron man 3  408992272  200000000  color       1   
2                    the hateful eight   54116191   44000000  color       0   
3                             margaret      46495   14000000  color       0   
4  the hobbit: the desolation of smaug  258355354  225000000  color       0   

   adventure  biography  
0          0          1  
1          1          0  
2          0          0  
3          0          0  
4          1          0  

[Hasil Akhir] Informasi Dataset Final:
<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 0 to 97
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   color                 80

Pada tahap akhir ini, dilakukan verifikasi dengan menampilkan lima baris pertama dari beberapa kolom kunci beserta kolom dummy genre yang baru dibuat untuk validasi visual. Selanjutnya, df.info() dan df.isnull().sum() dijalankan untuk memeriksa ringkasan teknis dan memastikan tidak ada lagi nilai kosong yang tersisa di seluruh dataset. Setelah semua pengecekan selesai, proses diakhiri dengan penyimpanan, di mana DataFrame yang sudah bersih dan final diekspor menjadi sebuah file CSV baru bernama movie_dataset_cleaned.csv ke lokasi yang telah ditentukan di Google Drive menggunakan df.to_csv(), dengan index=False untuk mencegah penulisan nomor indeks baris ke dalam file.